<a href="https://akademie.datamics.com/kursliste/">![title](bg_datamics_top.png)</a>

<center><em>© Datamics</em></center><br><center><em>Besuche uns für mehr Informationen auf <a href='https://akademie.datamics.com/kursliste/'>www.akademie.datamics.com</a></em></center>


# Aktienstimmung Analyse

Sieh Dir das Video an, um alle Details zu erfahren.

## Finaler Code
Hier ist der finale Code:

In [ ]:
# Dieser Abschnitt kann nur im Backtester importiert werden
from quantopian.algorithm import attach_pipeline, pipeline_output

# Allgemeine Pipeline-Importe
from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import AverageDollarVolume


# Verwende die kostenlose Probe in deinem Pipelinealgorithmus
from quantopian.pipeline.data.accern import alphaone_free


def initialize(context):
    # Stellen wir unsere rebalance-Funktion so ein, dass sie am Anfang jeder Woche ausgeführt wird
    schedule_function(my_rebalance, date_rules.every_day())

    
    attach_pipeline(make_pipeline(), "pipeline")

def make_pipeline():

    
    # Aussondern von Kleinaktien (penny stocks) und Wertpapieren mit geringer Liquidität
    dollar_volume = AverageDollarVolume(window_length=20)
    is_liquid = dollar_volume.rank(ascending=False) < 1000
 
    # Füge Pipelinefaktoren hinzu
    impact = alphaone_free.impact_score.latest
    sentiment = alphaone_free.article_sentiment.latest

    return Pipeline(columns={
            'impact': impact,
            'sentiment':sentiment,
            },
            screen = is_liquid)
            


    
def before_trading_start(context, data):
    port = pipeline_output('pipeline')
    
    # Erfasse Aktien mit 100 Einfluss und >0.5 Stimmung und verkaufe long.
    context.longs = port[(port['impact']==100) & (port['sentiment']>0.75)].index.tolist()
    
    # Erfasse Aktien mit 100 Einfluss und <-0.5 Stimmung und verkaufe long.
    context.shorts = port[(port['impact']==100) & (port['sentiment']< -0.75)].index.tolist()

    context.long_weight, context.short_weight = my_compute_weights(context)

def my_compute_weights(context):

    # Berechne gerade Zielgewichtungen für unsere long- und short-Positionen
    long_weight = 0.5 / len(context.longs)
    short_weight = -0.5 / len(context.shorts)

    return long_weight, short_weight

def my_rebalance(context, data):

    for security in context.portfolio.positions:
        if security not in context.longs and security not in context.shorts and data.can_trade(security):
            order_target_percent(security, 0)

    for security in context.longs:
        if data.can_trade(security):
            order_target_percent(security, context.long_weight)

    for security in context.shorts:
        if data.can_trade(security):
            order_target_percent(security, context.short_weight)
